In [2]:
import pandas as pd
import json
import re

In [3]:
cases = pd.read_csv("../extracted_cases.csv", encoding = "utf-8-sig")

In [4]:
old_cases = json.load(open('../data-old.json', encoding='utf-8'))

In [5]:
old_cases_df = pd.DataFrame(old_cases)

In [6]:
!pip install geopy

  Using cached https://files.pythonhosted.org/packages/53/fc/3d1b47e8e82ea12c25203929efb1b964918a77067a874b2c7631e2ec35ec/geopy-1.21.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/8b/62/26ec95a98ba64299163199e95ad1b0e34ad3f4e176e221c40245f211e425/geographiclib-1.50-py3-none-any.whl


You are using pip version 10.0.1, however version 20.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [7]:
from geopy.geocoders import Nominatim

geolocator = Nominatim()

c:\users\modestasj\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.21.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


In [8]:
geolocator.geocode("Lietuva belgu g. 5 klaipeda")

In [9]:
geolocator.geocode(f"Lietuva belgu g. 5 klaipeda")

KeyboardInterrupt: 

In [ ]:
import numpy as np

COUNTRY = 'Lietuva'
RETRIES = 3

def get_latitude(case):
    for i in range(RETRIES):
        try:
            return geolocator.geocode(case).latitude
        except:
            print(f"Failed: {case}")
        
    return np.nan

def get_longitude(case):
    for i in range(RETRIES):
        try:
            return geolocator.geocode(case).longitude
        except:
            print(f"Failed: {case}")
        
    return np.nan

cases['Latitude'] = cases.apply(lambda x: get_latitude(f"{x['addresses']}, {x['action_place_names']}, {x['city']}"), axis=1)
cases['Longitude'] = cases.apply(lambda x: get_longitude(f"{x['addresses']}, {x['action_place_names']}, {x['city']}"), axis=1)

In [ ]:
unique_df = old_cases_df.drop_duplicates('address')

In [ ]:
address_longitudes = {key: val for key, val in zip(unique_df['address'].values, unique_df['longitude'].values)}
address_latitudes = {key: val for key, val in zip(unique_df['address'].values, unique_df['latitude'].values)}
cases['latitude'] = cases.apply(lambda x: address_latitudes.get(re.sub(' +', ' ', x['addresses']), x['Latitude']), axis=1)
cases['longitude'] = cases.apply(lambda x: address_longitudes.get(re.sub(' +', ' ', x['addresses']), x['Longitude']), axis=1)

In [ ]:
cases['Time'] = cases.apply(lambda x: f"{x['dates']} {x['time']}" if not pd.isna(x['time']) else f"{x['dates']} 12:00", axis=1)

In [ ]:
cases.to_csv('ready_for_fix.csv', encoding='utf-8-sig')

In [ ]:
fixed_cases = pd.read_csv('ready_for_fix.csv', encoding='utf-8-sig')

In [ ]:
fixed_cases.dropna(subset=['case_id'])

In [ ]:
import pytz
from datetime import timedelta, datetime

local = pytz.timezone("EET")

local_dt = local.localize(datetime.today() + timedelta(hours=2), is_dst=None)
utc_dt = local_dt.astimezone(pytz.utc)

print(local_dt)
utc_dt

In [ ]:
fixed_cases = fixed_cases.dropna(subset=['case_id'])

In [ ]:
from datetime import datetime

t1 = datetime.timestamp(datetime.now())

datetime.fromtimestamp(t1)

fixed_cases["timestamp"] = pd.to_datetime(fixed_cases["Time"])
fixed_cases["timestamp"] = fixed_cases["timestamp"].apply(lambda x: local.localize(x, is_dst=None).astimezone(pytz.utc))

fixed_cases["timestamp"] = fixed_cases["timestamp"].apply(lambda x: datetime.timestamp(x))
#cases["timestamp"] = 
fixed_cases

In [ ]:
fixed_cases = fixed_cases.rename({"Unnamed: 0": "id"}, axis=1)
fixed_cases['id'] = np.arange(240, len(fixed_cases)+240)

In [ ]:
fixed_cases

In [ ]:
fixed_cases = fixed_cases[["id","addresses","action_place_names","city","Time","latitude","longitude","timestamp"]]

In [ ]:
fixed_cases.to_csv("../covid-cases-coordinates.csv", index=False)

In [ ]:
fixed_cases

In [ ]:
fixed_cases = fixed_cases.rename(columns={
    "addresses":"address",
    "action_place_names":"name",
    "city":"city",
    "Time":"time",
    "latitude":"latitude",
    "longitude":"longitude"
})
fixed_cases.to_json("../covid-cases-coordinates.json", orient="records")

In [107]:
import json

In [108]:
json.dumps(fixed_cases.to_dict(orient="records"), ensure_ascii=False)

'[{"id": 240, "addresses": "Oro uosto g. 4", "action_place_names": "Kauno oro uostas", "city": "Kaunas", "Time": "2/19/2020 12:00", "latitude": 54.967318000000006, "longitude": 24.073773000000003, "timestamp": 1582106400.0}, {"id": 241, "addresses": "Oro uosto g. 4", "action_place_names": "Kauno oro uostas", "city": "Kaunas", "Time": "3/14/2020 20:00", "latitude": 54.967318000000006, "longitude": 24.073773000000003, "timestamp": 1584208800.0}, {"id": 242, "addresses": "V. Kudirkos  g. 99", "action_place_names": "Respublikinė Šiaulių ligoninė", "city": "Šiauliai", "Time": "3/17/2020 19:00", "latitude": 55.94290225, "longitude": 23.3032079, "timestamp": 1584464400.0}, {"id": 243, "addresses": "Rodūnios kl. 10A", "action_place_names": "Vilniaus oro uostas", "city": "Vilnius", "Time": "3/13/2020 10:30", "latitude": 54.6430534, "longitude": 25.2810443, "timestamp": 1584088200.0}, {"id": 244, "addresses": "Vilniaus  g. 47B", "action_place_names": "Norfa", "city": "Joniškis", "Time": "3/14/20

In [88]:
fixed_cases.to_dict(orient="records", ensure_ascii=False)

TypeError: to_dict() got an unexpected keyword argument 'ensure_ascii'